<a href="https://colab.research.google.com/github/O-Kpy/Dacon/blob/main/Dacon_%E1%84%89%E1%85%A9%E1%84%89%E1%85%A5%E1%86%AF_%E1%84%8C%E1%85%A1%E1%86%A8%E1%84%80%E1%85%A1_%E1%84%87%E1%85%AE%E1%86%AB%E1%84%85%E1%85%B2_AI_%E1%84%80%E1%85%A7%E1%86%BC%E1%84%8C%E1%85%B5%E1%86%AB%E1%84%83%E1%85%A2%E1%84%92%E1%85%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install transformers

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split

import torch
from torch import nn
from torch import optim
from torch.optim import lr_scheduler
from torch.utils.data import DataLoader, Dataset, TensorDataset, SequentialSampler, random_split, RandomSampler


import transformers
from transformers import BertConfig, BertModel, BertTokenizer, AdamW, DistilBertModel, DistilBertTokenizer

In [8]:
train = pd.read_csv('/content/drive/MyDrive/dataset/Dacon/연습/소설 작가 분류 AI 경진대회_data/소설 작가 분류 AI 경진대회_data-train.csv', encoding='utf-8')
test = pd.read_csv('/content/drive/MyDrive/dataset/Dacon/연습/소설 작가 분류 AI 경진대회_data/소설 작가 분류 AI 경진대회_data-test_x.csv', encoding='utf-8')
submit = pd.read_csv('/content/drive/MyDrive/dataset/Dacon/연습/소설 작가 분류 AI 경진대회_data/소설 작가 분류 AI 경진대회_data-sample_submission.csv', encoding='utf-8')
train.shape, test.shape

((54879, 3), (19617, 2))

In [3]:
# 토큰화
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

sequence_len = 160
x_train_tokens = []
attention_mask = []

for text in train['text']:
  encoded_dict = tokenizer.encode_plus(text,
                                       add_special_tokens=True,
                                       max_length=sequence_len,
                                       padding='max_length',
                                       return_tensors='pt',
                                       truncation=True)

  x_train_tokens.append(encoded_dict['input_ids'])
  attention_mask.append(encoded_dict['attention_mask'])

bert = BertModel.from_pretrained('bert-base-uncased')

x_train_tokens = torch.cat(x_train_tokens, dim=0)
attention_mask = torch.cat(attention_mask, dim=0)
y_train = torch.tensor(train['author'])

x_train_tokens.size(), attention_mask.size(), y_train.size()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


(torch.Size([54879, 160]), torch.Size([54879, 160]), torch.Size([54879]))

In [4]:
# Data Load
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
batch_size = 16

dataset = TensorDataset(x_train_tokens.to(device),
                        attention_mask.to(device),
                        y_train.double().to(device))

train_size = int(len(dataset)*0.8)
val_size = len(dataset) - train_size

train_set, valid_set = random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_set,
                          sampler=RandomSampler(train_set),
                          batch_size=batch_size)
valid_loader = DataLoader(valid_set,
                          sampler=RandomSampler(valid_set),
                          batch_size=batch_size)

In [5]:
bert = BertModel.from_pretrained('bert-base-uncased')
bert.to(device)

example_batch = next(iter(train_loader))

print('what is drawn from our dataloader?', type(example_batch))
print('\nfirst entry:', example_batch[0].size(), type(example_batch[0]), example_batch[0].dtype)
print("\nsecond entry: ", example_batch[1].size(), type(example_batch[1]), example_batch[1].dtype)

batch_features = example_batch[0].to(device)
bert_output = bert(input_ids=batch_features)
print('bert output:', type(bert_output), len(bert_output))
print('first entry:', type(bert_output[0]), bert_output[0].size())
print('second entry:', type(bert_output[1]), bert_output[1].size())

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


what is drawn from our dataloader? <class 'list'>

first entry: torch.Size([16, 160]) <class 'torch.Tensor'> torch.int64

second entry:  torch.Size([16, 160]) <class 'torch.Tensor'> torch.int64
bert output: <class 'transformers.modeling_outputs.BaseModelOutputWithPoolingAndCrossAttentions'> 2
first entry: <class 'torch.Tensor'> torch.Size([16, 160, 768])
second entry: <class 'torch.Tensor'> torch.Size([16, 768])


In [6]:
PRE_TRAINED_MODEL_NAME = 'distilbert-base-cased'
tokenizer = DistilBertTokenizer.from_pretrained(PRE_TRAINED_MODEL_NAME)
bert_model = DistilBertModel.from_pretrained(PRE_TRAINED_MODEL_NAME)

bert_embeddings = bert_model.get_input_embeddings()

Some weights of the model checkpoint at distilbert-base-cased were not used when initializing DistilBertModel: ['vocab_layer_norm.weight', 'vocab_projector.bias', 'vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_transform.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [34]:
class TrainDataSet(Dataset):
    def __init__(self, excerpts, labels, tokenizer, max_len):
        self.excerpts = excerpts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.excerpts)
    
    def __getitem__(self, item):
        excerpt = str(self.excerpts[item])
        
        encoding  = self.tokenizer.encode_plus(
            excerpt,
            max_length = self.max_len,
            add_special_tokens=True,
            padding='max_length',
            return_attention_mask=True,
            return_token_type_ids=False,
            return_tensors='pt',
            truncation=True
        )
        
        return {
            'excerpt_text': excerpt,
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'targets': torch.tensor(self.labels[item], dtype=torch.long)
        }
class TestDataSet(Dataset):
    def __init__(self, ids, excerpts, tokenizer, max_len):
        self.ids = ids
        self.excerpts = excerpts
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.excerpts)
    
    def __getitem__(self, item):
        excerpt = str(self.excerpts[item])
        excerpt_id = str(self.ids[item])
        
        encoding  = self.tokenizer.encode_plus(
            excerpt,
            max_length = self.max_len,
            add_special_tokens=True,
            padding='max_length',
            return_attention_mask=True,
            return_token_type_ids=False,
            return_tensors='pt',
            truncation=True
        )
        
        return {
            'excerpt_id': excerpt_id,
            'excerpt_text': excerpt,
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
        }

def create_train_data_loader(df, tokenizer, max_len, batch_size):
    excerpts = df['text'].to_numpy(),
    print(f'Excerpts size: {len(excerpts)}')
    labels = df['author'].to_numpy(),
    dataset = TrainDataSet(excerpts=excerpts[0], labels=labels[0], tokenizer=tokenizer, max_len=max_len)
    print(f'Dataset size: {len(dataset)}')
    return DataLoader(dataset, batch_size=batch_size, num_workers=4, shuffle=True)

def create_test_data_loader(df, tokenizer, max_len, batch_size):
    excerpts = df['text'].to_numpy(),
    ids = df['index'].to_numpy(),
    print(f'Excerpts size: {len(excerpts)}')
    dataset = TestDataSet(ids= ids[0], excerpts=excerpts[0], tokenizer=tokenizer, max_len=max_len)
    print(f'Dataset size: {len(dataset)}')
    return DataLoader(dataset, batch_size=batch_size, num_workers=4)


train_set, val_set = train_test_split(train, test_size=0.2)

BATCH_SIZE = 16
MAX_LEN = 160

train_data_loader = create_train_data_loader(train_set, tokenizer, max_len=MAX_LEN, batch_size=BATCH_SIZE)
val_data_loader = create_train_data_loader(val_set, tokenizer, max_len=MAX_LEN, batch_size=BATCH_SIZE)

device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

Excerpts size: 1
Dataset size: 43903
Excerpts size: 1
Dataset size: 10976


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [13]:
class distilbertclassifier(nn.Module):
  def __init__(self):
    super(distilbertclassifier, self).__init__()
    self.num_labels = train['author'].nunique()
    self.softmax = nn.Softmax(dim=1)
    self.distilbert = DistilBertModel.from_pretrained('distilbert-base-cased')
    self.classifier = nn.Linear(self.distilbert.config.dim, train['author'].nunique())
    self.dropout = nn.Dropout(0.3)

    nn.init.xavier_normal_(self.classifier.weight)

  def forward(self, input_ids, attention_mask):
    distilbert_output = self.distilbert(input_ids=input_ids,
                                        attention_mask=attention_mask)
    hidden_state = distilbert_output[0]
    pooled_output = hidden_state[:,0,:]
    pooled_output = self.dropout(pooled_output)
    logits = self.classifier(pooled_output)
    return logits

In [14]:
model = distilbertclassifier()
model = model.to(device)

Some weights of the model checkpoint at distilbert-base-cased were not used when initializing DistilBertModel: ['vocab_layer_norm.weight', 'vocab_projector.bias', 'vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_transform.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [15]:
model.distilbert.config

DistilBertConfig {
  "_name_or_path": "distilbert-base-cased",
  "activation": "gelu",
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "output_past": true,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.9.0",
  "vocab_size": 28996
}

# Training

In [24]:
epochs = 3
if torch.cuda.is_available():
  optimizer = transformers.AdamW(model.parameters(), lr=2e-5, correct_bias=False)
  total_steps = len(train_data_loader) * epochs
  scheduler = transformers.get_linear_schedule_with_warmup(
        optimizer,
        num_warmup_steps=0,
        num_training_steps=total_steps
    )
  loss_fn = nn.CrossEntropyLoss().to(device)

In [29]:
def train_epoch(model, data_loader, loss_fn, optimizer, scheduler, n_examples):
  model = model.train()
  losses = []
  correct_pred = 0

  for d in data_loader:
    input_ids = d['input_ids'].to(device)
    attention_mask = d['attention_mask'].to(device)
    targets = d['targets'].to(device)

    optimizer.zero_grad()

    outputs = model(input_ids=input_ids, attention_mask=attention_mask)
    _, preds = torch.max(outputs, dim=1)

    loss = loss_fn(outputs, targets)
    correct_pred += torch.sum(preds==targets)
    losses.append(loss.item())
    loss.backward()

    optimizer.step()
    scheduler.step()
  return correct_pred.double() / n_examples, np.mean(losses)

In [30]:
def eval_model(model, data_loader, loss_fn, n_examples):
  model = model.eval()
  losses = []
  correct_preds = 0

  with torch.no_grad():
    for d in data_loader:
      input_ids = d['input_ids'].to(device)
      attention_mask = d['attention_mask'].to(device)
      targets = d['targets'].to(device)

      outputs = model(input_ids=input_ids, attention_mask=attention_mask)
      _, preds = torch.max(outputs, dim=1)
      loss = loss_fn(outputs, targets)
      correct_preds += torch.sum(preds == targets)
      losses.append(loss.item())
  return correct_preds/ n_examples, np.mean(losses)

In [22]:
def predict(model, data_loader, submission_df):
  model = model.eval()
  with torch.no_grad():
    for d in data_loader:
      input_ids = d['input_ids'].to(device)
      attention_mask = d['attention_mask'].to(device)
      outputs = model(input_ids=input_ids, attention_mask=attention_mask)
      print(outputs)

In [31]:
filename = 'final.pt'

model = model.to(device)
history={}
best_accuracy = 0

for epoch in range(epochs):
  print(f'Epoch {epoch + 1}/{epochs}')
  print('-'*10)

  train_acc, train_loss = train_epoch(model=model, data_loader=train_data_loader,
                                      loss_fn=loss_fn,
                                      optimizer=optimizer,
                                      scheduler=scheduler,
                                      n_examples=len(train_set))
  print(f'Train loss:{train_loss}, accuracy:{train_acc}')

  val_acc, val_loss = eval_model(model, val_data_loader,
                                 loss_fn,
                                 n_examples=len(val_set))
  print(f'Validation loss:{val_loss}, accuracy:{val_acc}')

torch.save(model.state_dict(), filename)

Epoch 1/3
----------


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Train loss:0.22355468708024978, accuracy:0.9245154089697744
Validation loss:0.19091577128623952, accuracy:0.9392310976982117
Epoch 2/3
----------
Train loss:0.22652045020478256, accuracy:0.9240143042616679
Validation loss:0.1909157702179677, accuracy:0.9392310976982117
Epoch 3/3
----------
Train loss:0.22444245059532927, accuracy:0.9246976288636313
Validation loss:0.19091577061320283, accuracy:0.9392310976982117


In [110]:
test_data_loader = create_test_data_loader(test, tokenizer, max_len=MAX_LEN, batch_size=BATCH_SIZE)

sample = next(iter(test_data_loader))

input_ids = sample['input_ids'].to(device)
attention_mask = sample['attention_mask'].to(device)

print(input_ids.shape)
print(attention_mask.shape)
prob, pred = torch.max(model(input_ids=input_ids, attention_mask=attention_mask),dim=1)
print(prob)
print(pred)

Excerpts size: 1
Dataset size: 19617


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


torch.Size([16, 160])
torch.Size([16, 160])
tensor([ 5.0812,  7.6404, 10.3209,  7.4818,  6.9561, 10.4946,  7.0634,  3.1641,
        10.9128,  6.6594, 12.2653,  7.9196, 12.5967, 10.6816, 12.0638,  6.2681],
       device='cuda:0', grad_fn=<MaxBackward0>)
tensor([3, 1, 0, 2, 0, 3, 3, 1, 0, 0, 3, 4, 1, 2, 2, 0], device='cuda:0')


In [122]:
def test_model(model, data_loader, results_df):
  model = model.eval()
  submission = []

  with torch.no_grad():
    for d in data_loader:
      excerpt_ids = d['excerpt_id']
      input_ids = d['input_ids'].to(device)
      attention_mask = d['attention_mask'].to(device)

      outputs = model(input_ids=input_ids, attention_mask=attention_mask)
      outputs = nn.functional.softmax(outputs, dim=1)
      for i, excerpt_id in enumerate(excerpt_ids):
        results_df.loc[results_df['index'] == excerpt_id, ['0','1','2','3','4']] =  outputs[i].tolist()

  return results_df

# 마지막 proba 얻기

In [112]:
submit = pd.read_csv('/content/drive/MyDrive/dataset/Dacon/연습/소설 작가 분류 AI 경진대회_data/소설 작가 분류 AI 경진대회_data-sample_submission.csv', encoding='utf-8')

In [123]:
results_df = test_model(model, test_data_loader, submit)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [120]:
output = model(input_ids, attention_mask)
nn.functional.softmax(output, dim=1)

tensor([[1.1774e-03, 2.9400e-01, 1.9017e-02, 6.8580e-01, 4.2090e-06],
        [5.1597e-05, 9.9913e-01, 2.0068e-06, 4.8066e-05, 7.7074e-04],
        [9.9999e-01, 1.1113e-06, 1.0914e-07, 2.0152e-06, 2.3305e-06],
        [1.7971e-05, 1.0480e-02, 9.8948e-01, 2.0416e-05, 3.8130e-06],
        [9.9574e-01, 3.9873e-03, 1.4071e-05, 9.7049e-05, 1.5680e-04],
        [1.0702e-06, 2.0752e-06, 1.2159e-05, 9.9996e-01, 1.9981e-05],
        [1.7768e-05, 6.2105e-05, 4.3146e-04, 9.9917e-01, 3.2071e-04],
        [4.5732e-01, 5.3310e-01, 2.4002e-04, 1.1065e-04, 9.2321e-03],
        [9.9999e-01, 5.5559e-06, 6.0840e-06, 1.2717e-07, 2.1647e-07],
        [9.9157e-01, 7.0728e-03, 5.2034e-06, 1.3035e-03, 4.3497e-05],
        [3.6595e-07, 5.9945e-07, 1.1002e-06, 1.0000e+00, 2.8058e-07],
        [1.8181e-04, 5.4512e-06, 2.0667e-05, 4.9757e-06, 9.9979e-01],
        [1.7410e-06, 1.0000e+00, 2.2274e-08, 6.9940e-08, 1.0018e-07],
        [2.5895e-08, 4.6673e-04, 9.9951e-01, 6.6918e-09, 2.6639e-05],
        [8.2277e-09,